In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import torchvision.models as models

from skimage import io, transform
import numpy as np
import pandas as pd
import os
from PIL import Image
import random

In [2]:
NUM_EPOCHS = 10
LR = 1e-2
BATCH_SIZE = 4

In [3]:
tfms = transforms.Compose([transforms.ToTensor()])

In [4]:
categories = pd.read_csv("/home/krypt/myStuff/skillenza/paniithackathon2019/training.5k/training/solution.csv")["category"]
def getitem(idx):
    img_name = os.path.join("/home/krypt/myStuff/skillenza/paniithackathon2019/training.5k/training/training/", str(idx+1) + ".png")
    image = Image.open(img_name)
    if tfms:
        image = tfms(image)
    category = categories[idx]
    sample = {'image': image, 'category': category}

    return sample

In [5]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [6]:
model = models.resnet34(pretrained=True)

In [7]:
model = nn.Sequential(*list(model.children())[:-2],
                     nn.Conv2d(512, 6, 3, 1),
                     nn.AdaptiveAvgPool2d(1), Flatten(),
                     nn.Softmax())

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.95)

In [8]:
print(model.children)
count = 0
for param in model.parameters():
    param.requires_grad = False
    count += 1
    if count >= 8:
        break

<bound method Module.children of Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)


In [ ]:
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    accuracy = 0
    count = 0
    for c in range(1,5000):
        optimizer.zero_grad()

        data = getitem(random.randint(0,4999))
        
        images = data["image"].unsqueeze(0)
        targets = torch.tensor([np.asscalar(data["category"])]).long()-1
        
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        
        outs = model(images)
        print(outs)
        print(targets)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
        accuracy += torch.sum(torch.argmax(outs, dim=1) == targets)
        count += data["image"].size(0)

        if c%200 == 0:
            print("Epoch:", epoch, "Iter:", c, "Total loss:", total_loss, "Accuracy:", accuracy.cpu().numpy()/count)
                
    print("Epoch:", epoch, "Total loss:", total_loss)
    print("Epoch:", epoch, "Accuracy:", accuracy.cpu().numpy()/count)    

/home/krypt/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([2], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([4], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([4], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([5], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([5], device='cuda:0')
tensor([[0., 0., 0., 0., 0.,